## Emily: please run this notebook for all sessions and subjects
- This notebook resamples the functional and mask images to the atlas space
- The atlas (from templateflow) is the Schaefer 2018 1000 parcels 17 networks atlas, but we will want to make this a parameter in the future

In [1]:
from nilearn.image import resample_img
import nibabel as nib
from templateflow import api
import numpy as np
import os
import glob

In [2]:
## Specify subject and session here
subject_str = "MATHAI"
session_str = "SHAM"

In [3]:
## Specify the paths here -- you will need to change this to your own paths
path_to_func = f"/Users/jacekdmochowski/PROJECTS/fus/data/fmriprep_out/sub-{subject_str}/ses-{session_str}/func"
path_to_anat = f"/Users/jacekdmochowski/PROJECTS/fus/data/fmriprep_out/sub-{subject_str}/ses-{session_str}/anat"

## Find the filenames of the anatomical, functional, and mask images
matching_anats = glob.glob(os.path.join(path_to_anat, '*space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'))
matching_funcs = glob.glob(os.path.join(path_to_func, '*space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'))
matching_masks = glob.glob(os.path.join(path_to_func, '*space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'))

if len(matching_anats) == 1:
    anat_filename = matching_anats[0]
else:
    raise ValueError("No valid anatomical")

if len(matching_funcs) == 1:
    bold_filename = matching_funcs[0]
else:
    raise ValueError("No valid functional")

if len(matching_masks) == 1:
    mask_filename = matching_masks[0]
else:
    raise ValueError("No valid mask")

## Load the anatomical, bold and mask data
anat = nib.load(os.path.join(path_to_anat, anat_filename))
bold = nib.load(os.path.join(path_to_func, bold_filename))
mask = nib.load(os.path.join(path_to_func, mask_filename))

# load or generate resampled_bold and resampled_mask to disk
path_to_save_resampled_bold = os.path.join(path_to_func,
                                        f"sub-{subject_str}_ses-{session_str}_task-prefuspost_space-Schaefer2018_desc-preproc_bold_resampled.nii.gz")

path_to_save_resampled_mask = os.path.join(path_to_func,
                                        f"sub-{subject_str}_ses-{session_str}_task-prefuspost_space-Schaefer2018_desc-brain_mask_resampled.nii.gz")

## get atlas -- TODO: make the atlas a parameter
path = api.get('MNI152NLin2009cAsym', atlas="DiFuMo", desc="1024dimensions", resolution=2, suffix="probseg",
               extension="nii.gz")

#path = api.get('MNI152NLin2009cAsym', atlas="Schaefer2018", desc="1000Parcels17Networks", resolution=2, suffix="dseg",
#               extension="nii.gz")

atlas = nib.load(path)
target_affine = atlas.affine
target_shape = np.array(atlas.dataobj).shape

# get the atlas labels
atlas_labels = np.unique(np.array(atlas.dataobj))

In [4]:
## resample the bold and mask to the atlas
if os.path.exists(path_to_save_resampled_bold):
    resampled_bold = nib.load(path_to_save_resampled_bold)
else:
    resampled_bold = resample_img(bold, target_affine=target_affine, target_shape=target_shape[0:3])
    nib.save(resampled_bold, path_to_save_resampled_bold)

if os.path.exists(path_to_save_resampled_mask):
    resampled_mask = nib.load(path_to_save_resampled_mask)
else:
    resampled_mask = resample_img(mask, target_affine=target_affine, target_shape=target_shape[0:3],
                                  interpolation='nearest')
    nib.save(resampled_mask, path_to_save_resampled_mask)